# Airborne Time-Domain

This type of survey can be used to store airborne time-domain electromagnetic (ATEM) data defined by a fixed transmitter-receiver loop configuration. The survey is made up of two entities ([AirborneTEMTransmitters](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.airborne_tem.AirborneTEMTransmitters) and [AirborneTEMReceivers](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.airborne_tem.AirborneTEMReceivers)) linked by their metadata.

The following example shows how to generate an airborne TEM survey with associated data stored in `geoh5` format and accessible from [Geoscience ANALYST](https://mirageoscience.com/mining-industry-software/geoscience-analyst/).

![atemSurvey](./images/atem_survey.png)

In [ ]:
import numpy as np

from geoh5py.objects import AirborneTEMReceivers, AirborneTEMTransmitters
from geoh5py.workspace import Workspace


# Create a new project
workspace = Workspace("my_project.geoh5")

# Define the pole locations
n_stations = 9
n_lines = 2
x_loc, y_loc = np.meshgrid(
    np.linspace(0, 60, n_stations), np.linspace(-20, 20.0, n_lines)
)
vertices = np.c_[x_loc.ravel(), y_loc.ravel(), np.zeros_like(x_loc).ravel()]

# Assign a line ID to the poles (vertices)
parts = np.kron(np.arange(n_lines), np.ones(n_stations)).astype("int")

# Create the survey as a coincident loop system
aem_receivers = AirborneTEMReceivers.create(workspace, vertices=vertices, parts=parts)
aem_transmitters = AirborneTEMTransmitters.create(
    workspace, vertices=vertices, parts=parts
)

We have so far created two seperate entities, one for transmitter locations and another for the receivers. In order to finalize the survey, the association must be made between the two entities:

In [ ]:
aem_receivers.transmitters = aem_transmitters

or equivalently

In [ ]:
aem_transmitters.receivers = aem_receivers

Only one of the two options above is needed. 

Once linked, the two entities will share changes applied to the metadata. For example, changing the `input_type` property on the transmitters yield:

In [ ]:
aem_transmitters.input_type = "Tx and Rx"
print(aem_receivers.input_type)

## Metadata

Along with the survey object itself, the metadata contains all the necessary information to define the geophysical experiment.

In [ ]:
aem_receivers.metadata

### Channels

List of time channels at which the data are provided.

In [ ]:
aem_receivers.channels = np.logspace(-5, -2, 10)  # Simple sweep from 1 to 10 ms

### Input type

Label defining how the survey was created.

- `Rx`: Survey defined from the `AirborneTEMReceivers` positions, with the`AirborneTEMTransmitters` added from offsets.
- `Tx`: Survey defined from the `AirborneTEMTransmitters` position, with the`AirborneTEMReceivers` added from offsets.
- `Tx and Rx`: Survey defined by both the `AirborneTEMTransmitters` and the`AirborneTEMReceivers` positions.

### Property groups

List of [PropertyGroup](../api/geoh5py.groups.rst#module-geoh5py.groups.property_group)s defining the various data components (e.g. `dBzdt`, `Bz`, ...). It is expected that each component contains data channels at all times and in the same order as defined in `Channels`.

The class method [add_component_data](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.add_components_data) can help users add data from nested dictionaries. Below is an example using four components:

In [ ]:
# Create some simple data
def data_fun(t):
    1.0 / t * np.sin(np.pi * (x_loc * y_loc).ravel() / 800.0)

# Create a nested dictionary of time data.
data = {
    "dBdt": {
        f"time[{tt}]": {"values": data_fun(time)}
        for tt, time in enumerate(aem_receivers.channels)
    }
}

aem_receivers.add_components_data(data)

Metadata are also updated to reflect the addition of component data.

In [ ]:
aem_receivers.metadata

Data channels associated with each component can be quickly accessed through the [BaseEMSurvey.components](../api/geoh5py.objects.surveys.electromagnetics.rst#geoh5py.objects.surveys.electromagnetics.base.BaseEMSurvey.components) property:

In [ ]:
aem_receivers.components["dBdt"]

### Receivers

Generic label used for surveys to identify the receiver entity. References to itself in the case of `AirborneTEMReceivers`.

### Survey type

Static label identifier for `Airborne TEM` survey type.

### Transmitters

Generic label used for surveys to identify the transmitter entity. References to itself in the case of `AirborneTEMTransmitters`.

### Unit

Units for time sampling of the data - must be one of `Seconds (s)`, `Milliseconds (ms)`, `Microseconds (us)` or `Nanoseconds (ns)`.

### Loop radius

Specifies the transmitter loop radius.

### Custom fields

`Metadata` are stored in `geoh5` as a `json` structure allowing for custom data fields to be added to the survey.  Information such as flight data, date/time, offsets, etc. can be added as `string`, `float` and `int`.

In [ ]:
aem_receivers.edit_metadata({"Weather": "sunny"})

![atem_custom](./images/atem_metadata_custom.png)

Aternatively, a `uuid.UUID` value can be used if the information is to be provided at every survey position.

In [ ]:
# Add a new data entry
abc = aem_receivers.add_data(
    {"abc": {"values": np.random.randn(aem_receivers.n_vertices)}}
)

# Assign the data as 'Weather' metadata
aem_receivers.edit_metadata({"Weather": abc.uid})

`Geoscience ANALYST` will automatically create a link referencing the data field to the entity in the project tree.

![atem_uid](./images/atem_metadata_uid.png)

#### Reserved keywords

For known metadata, such as flight dynamics (`yaw`, `pitch`, `roll`) and offsets (`inline`, `crossline`, `vertical`) the suffix `property` and `value` will get replaced based on the input value:

In [ ]:
aem_receivers.yaw = 15.0

![atem_yaw_value](./images/atem_metadata_yaw_value.png)

In [ ]:
aem_receivers.yaw = abc.uid  # Assign to the yaw property

![atem_yaw_property](./images/atem_metadata_yaw_property.png)

In [ ]:
workspace.close()